In [ ]:
# Only run this once to install the necessary packages
%pip install torch
%pip install pandas
%pip install sklearn
%pip install scipy

In [ ]:
import torch
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.preprocessing import normalize
from IPython.display import clear_output

In [ ]:
data = pd.read_csv("books_data.csv", nrows=40000)
data['Title'] = data['Title'].fillna('Unknown')
data['categories'] = data['categories'].fillna('Unknown')
data['description'] = data['description'].fillna('')
data['description'] = data['description'].apply(lambda x: x.lower())
data['book_content'] = (
    (data['Title'] + ' ') * 2
    + data['description'] + ' '
    + data['authors'].apply(lambda x: ' '.join(x) if isinstance(x, list) else '') + ' '
    + data['categories'].apply(lambda x: ' '.join(x) * 5 if isinstance(x, list) else '')
)
data['book_content'] = data['book_content'].str.replace(r'[^\w\s]', '', regex=True).str.lower()

In [ ]:
book_embeddings = torch.load('bert_embeddings.pt')
print(book_embeddings.shape)

In [ ]:
normalized_book_embeddings = normalize(book_embeddings)

In [ ]:
euclidean_dist_matrix = cdist(normalized_book_embeddings, normalized_book_embeddings, metric='euclidean')

In [ ]:
def recommend_books_by_euclidean(book_title, threshold, euclidean_dist_matrix):
    idx = data[data['Title'] == book_title].index[0]
    
    # Compute the Euclidean distance scores
    dist_scores = list(enumerate(euclidean_dist_matrix[idx]))
    
    # Sort the books based on Euclidean distance (lower distance means more similar)
    dist_scores = sorted(dist_scores, key=lambda x: x[1], reverse=False)
    
    # Filter recommendations based on the threshold (optional)
    recommendations = [(data['Title'].iloc[i], "{:.5f}".format(score)) 
                       for i, score in dist_scores if score <= threshold]
    
    return recommendations

In [ ]:
book_title = ''
while (book_title != 'q'):
  clear_output(wait=True)
  book_title = input("Enter the title of a book: ")
  recommended_books = recommend_books_by_euclidean(book_title, 1.0, euclidean_dist_matrix)
  f = open('euclidean_output.txt', 'w')
  f.write('Counts: ' + str(len(recommended_books)) + '\n\n')
  for book in recommended_books:
    f.write(book[1] + ' | ' + str(book[0]) + '\n')

  print('Found: ' + str(len(recommended_books)))